In [1]:
import numpy as np

import os
os.environ['pRT_input_data_path'] = '/pscratch/sd/m/mkaraman/input_data'
os.environ["OMP_NUM_THREADS"] = "1"

from petitRADTRANS import Radtrans
from petitRADTRANS import nat_cst as nc 
from petitRADTRANS.retrieval.parameter import Parameter
from petitRADTRANS.retrieval.models import emission_model_diseq

In [2]:
# Load scattering version of pRT
atmosphere = Radtrans(line_species = ['H2O_HITEMP',
                                      'CO_all_iso_HITEMP',
                                      'CH4',
                                      'NH3',
                                      'CO2',
                                      'H2S',
                                      'VO',
                                      'TiO_all_Exomol', 
                                       #'FeH',
                                      'PH3',
                                      'Na_allard',
                                      'K_allard'],
                      cloud_species = ['MgSiO3(c)_cd',"Fe(c)_cd"],
                      rayleigh_species = ['H2', 'He'],
                      continuum_opacities = ['H2-H2', 'H2-He'],
                      wlen_bords_micron = [0.95, 2.45],
                      do_scat_emis = True)

pressures = np.logspace(-6, 2, 154)
atmosphere.setup_opa_structure(pressures)

def Simulator(params): 

    '''
Dictionary of required parameters:
                *  D_pl : Distance to the planet in [cm]
                *  log_g : Log of surface gravity
                *  R_pl : planet radius [cm]
                *  T_int : Interior temperature of the planet [K]
                *  T3 : Innermost temperature spline [K]
                *  T2 : Middle temperature spline [K]
                *  T1 : Outer temperature spline [K]
                *  alpha : power law index in tau = delta * press_cgs**alpha
                *  log_delta : proportionality factor in tau = delta * press_cgs**alpha
                *  sigma_lnorm : Width of cloud particle size distribution (log normal)
                *  log_pquench : Pressure at which CO, CH4 and H2O abundances become vertically constant
                *  Fe/H : Metallicity
                *  C/O : Carbon to oxygen ratio
                *  log_kzz : Vertical mixing parameter
                *  fsed : sedimentation parameter
                *  log_X_cb : Scaling factor for equilibrium cloud abundances.
                
                
Parameter  Value         Parameter                     Value
T 1        330.6 K       log(X_0 Fe /X_eq Fe)          -0.86
T 2        484.7 K       log(X_0 MgSiO3 /X_eq MgSiO3)  -0.65
T 3        687.6 K       fsed                           3
log(δ)     -7.51         log(K zz /cm 2 s −1)           8.5
α          1.39          σg                             2
T 0        1063.6 K      R_P                            1 R J
C/O        0.55          log(g/cm s −2)                 3.75
[Fe/H]     0             log(P quench/bar)             -10 

C/O, Fe/H, log_pquench, XFe, XMgSiO3, fsed, log_kzz, sigma_lnorm, log_g, R_pl,
T_int, T3, T2, T1, alpha, log_delta- Molliere

C/O, Fe/H, log_pquench, XFe, XMgSiO3, log_g, R_pl,
T_int, T3, T2, T1, alpha, log_delta - me

#change the T3, T2, T1 and log delta values in models.py to

T1 = parameters['T1'].value                        ########################new
T2 = parameters['T2'].value                        ########################new
T3 = parameters['T3'].value                        ########################new
delta = 10**parameters['log_delta'].value          ########################new

before running the TABULATED DATA in the paper M2020

'''
    
    #16 params for simulation.
    
    xCO = params[0]                 # 0.55
    FeH = params[1]                # 0.
    log_pquench = params[2]        # -10.
    XFe = params[3]                # -0.86
    XMgSiO3 = params[4]            # -0.65
    fsed = params[5]               # 3. 
    log_kzz = params[6]            # 8.5
    sigma_lnorm = params[7]        # 2.
    log_g = params[8]              # 3.75
    R_pl = params[9]               # 1
    T_int = params[10]             # 1063.6
    xT3 = params[11]                # 687.6 
    xT2 = params[12]                # 484.7 
    xT1 = params[13]                # 330.6 
    alpha = params[14]             # 1.39
    log_delta = params[15]         # -7.51

    #print(params)
        
    parameters={}
    parameters['D_pl'] = Parameter(name = 'D_pl', is_free_parameter = False, value = 41.2925*nc.pc) 
    parameters['log_g'] = Parameter(name ='log_g',is_free_parameter = False, value = log_g)
    parameters['R_pl'] = Parameter(name = 'R_pl', is_free_parameter = False, value = R_pl* nc.r_jup_mean)
    parameters['T_int'] = Parameter(name ='T_int',is_free_parameter = False, value = T_int)
    parameters['T3'] = Parameter(name = 'T3', is_free_parameter = False, value = xT3)
    parameters['T2'] = Parameter(name ='T2',is_free_parameter = False, value = xT2)
    parameters['T1'] = Parameter(name = 'T1', is_free_parameter = False, value = xT1)
    parameters['alpha'] = Parameter(name ='alpha',is_free_parameter = False, value = alpha)
    parameters['log_delta'] = Parameter(name ='log_delta',is_free_parameter = False, value = log_delta)
    parameters['sigma_lnorm'] = Parameter(name ='sigma_lnorm',is_free_parameter = False, value = sigma_lnorm)
    parameters['log_pquench'] = Parameter(name ='log_pquench',is_free_parameter = False, value = log_pquench)
    parameters['Fe/H'] = Parameter(name ='Fe/H',is_free_parameter = False, value = FeH)
    parameters['C/O'] = Parameter(name ='C/O',is_free_parameter = False, value = xCO)
    parameters['log_kzz'] = Parameter(name ='log_kzz',is_free_parameter = False, value = log_kzz)
    parameters['fsed'] = Parameter(name ='fsed',is_free_parameter = False, value = fsed)
    parameters['log_X_cb'+ '_Fe(c)'] = Parameter(name ='log_X_cb'+'_Fe(c)',is_free_parameter = False, value = XFe)
    parameters['log_X_cb'+'_MgSiO3(c)'] = Parameter(name ='log_X_cb'+'_MgSiO3(c)',is_free_parameter = False, value = XMgSiO3)
    parameters['pressure_scaling'] = Parameter(name ='pressure_scaling',is_free_parameter = False, value = 10)
    parameters['pressure_width'] = Parameter(name ='pressure_width',is_free_parameter = False, value = 3)
    parameters['pressure_simple'] = Parameter(name ='pressure_simple',is_free_parameter = False, value = 100)
    
    return emission_model_diseq(atmosphere, parameters, AMR = True)

/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:119: FutureWarning: pRT_input_data_path was set by an environment variable. In a future update, the path to the petitRADTRANS input_data will be set within a .ini file that will be automatically generated into the user home directory (OS agnostic), inside a .petitradtrans directory
  warnings.warn(f"pRT_input_data_path was set by an environment variable. In a future update, the path to "


Emission scattering is enabled: enforcing test_ck_shuffle_comp = True
  Read line opacities of H2O_HITEMP...
  Read line opacities of CO_all_iso_HITEMP...
  Read line opacities of CH4...
 Done.
  Read line opacities of NH3...
 Done.
  Read line opacities of CO2...
 Done.
  Read line opacities of H2S...
 Done.
  Read line opacities of VO...
 Done.
  Read line opacities of TiO_all_Exomol...
  Read line opacities of PH3...
 Done.
  Read line opacities of Na_allard...
  Read line opacities of K_allard...

  Read in opacity of cloud species MgSiO3 ...
  Read in opacity of cloud species Fe ...
  Read CIA opacities for H2-H2...
  Read CIA opacities for H2-He...
Done.



In [3]:
low = np.array([0.1, -1.5, -6.0, -3.5, -3.5, 1.0, 5.0, 1.05, 2.0, 0.7, 300.0, 0., 0., 0.,1., 0. ])
high = np.array([1.6, 1.5, 3.0, 4.5, 4.5, 11.0, 13.0, 3.0, 5.5, 2.0, 2300.0, 1., 1., 1., 2., 1. ])

sigma = 1.25754e-17 #* 1e16

In [ ]:
seeds = np.arange(70,100)

for seed in seeds:
    n_prior_samples = 12800
    np.random.seed(seed)
    prior_samples = np.random.uniform(low, high, size = (n_prior_samples,16))

    from multiprocess import Pool
    import time

    print("Start!", seed)

    t0 = time.time()
    with Pool(128) as p:
        obs = list(p.map(Simulator, prior_samples))
    print(time.time()-t0)

    data = np.array(obs)[:,1,:]

    np.save("output/atm_retr_data_12800_"+str(seed)+".npy", data)
    np.save("output/atm_retr_params_12800_"+str(seed)+".npy", prior_samples)

    print("Done!", seed)

Start! 70


/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2181: RuntimeWarning: divide by zero encountered in divide
  3.0
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2188: RuntimeWarning: divide by zero encountered in log
  diff = np.log(cloud_radii[:, None, None]) - np.log(r_g)
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2190: RuntimeWarning: invalid value encountered in multiply
  n


Region of zero opacity detected, setting the photon destruction probability in this spectral range to 1.


/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:1391: UserWarning: Cloud rescaling lead to nan opacities, skipping RT calculation!
  warnings.warn("Cloud rescaling lead to nan opacities, skipping RT calculation!")
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2181: RuntimeWarning: divide by zero encountered in divide
  3.0
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2190: RuntimeWarning: invalid value encountered in multiply
  n


Region of zero opacity detected, setting the photon destruction probability in this spectral range to 1.


...

/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2184: RuntimeWarning: overflow encountered in power
  / (4.0 * np.pi * rho_p * (r_g ** 3))
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2184: RuntimeWarning: overflow encountered in power
  / (4.0 * np.pi * rho_p * (r_g ** 3))
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2181: RuntimeWarning: invalid value encountered in divide
  3.0


Region of zero opacity detected, setting the photon destruction probability in this spectral range to 1.


/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:1391: UserWarning: Cloud rescaling lead to nan opacities, skipping RT calculation!
  warnings.warn("Cloud rescaling lead to nan opacities, skipping RT calculation!")


391.45222997665405
Done! 75
Start! 76


/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2184: RuntimeWarning: overflow encountered in power
  / (4.0 * np.pi * rho_p * (r_g ** 3))
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2184: RuntimeWarning: overflow encountered in power
  / (4.0 * np.pi * rho_p * (r_g ** 3))
/global/u2/m/mkaraman/petitRADTRANS/petitRADTRANS/radtrans.py:2184: RuntimeWarning: overflow encountered in power
  / (4.0 * np.pi * rho_p * (r_g ** 3))
